# Mitigating Unwanted Biases with Adversarial Learning

This notebook is a guide to the paper ([archiv](https://arxiv.org/pdf/1801.07593.pdf))

    Brian Zhang, Blake Lemoine and Margaret Mitchell. Mitigating Unwanted Biases with Adversarial Learning. AAAI Conference on AI, Ethics and Society, 2018.

This notebook is adapted from [here](https://colab.research.google.com/notebooks/ml_fairness/adversarial_debiasing.ipynb#scrollTo=1x0PRJii9C33) and made compatible with Python 3.

In [1]:
!pip install -U gensim~=3.2.0

Requirement already up-to-date: gensim~=3.2.0 in c:\programdata\anaconda3\lib\site-packages (3.2.0)


In [1]:
import gensim
import gzip
import numpy as np
import os
import pandas as pd
import tensorflow as tf
import gensim.downloader as gs_api

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


# Load and Process Data

Download the Google News word2vec dataset from [here](https://drive.google.com/uc?id=0B7XkCwpI5KDYNlNUTTlSS21pQmM&export=download). The dataset contains about 100 billion words and the model contains 300-dimensional vectors for 3 million words and phrases [source](https://code.google.com/archive/p/word2vec/).

Gensim data can be found [here](https://github.com/RaRe-Technologies/gensim-data).

Also download the Google analogy dataset from [here](https://github.com/nicholas-leonard/word2vec/blob/master/questions-words.txt). It contains 19557 question pairs.

In [24]:
%%time
client = gs_api.load("glove-wiki-gigaword-300")

C:\ProgramData\Anaconda3\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


Wall time: 2min 26s


In [2]:
data_dir = 'C:/Users/ajthampi/Workspace/Data/Interpretability/'
WORD2VEC_FILE = os.path.join(data_dir, "GoogleNews-vectors-negative300.bin.gz")
ANALOGIES_FILE = os.path.join(data_dir, "questions-words.txt")

In [3]:
def load_word2vec_format(f, max_num_words=None):
    """Loads word2vec data from a file handle.

    Similar to gensim.models.keyedvectors.KeyedVectors.load_word2vec_format
    but takes a file handle as input rather than a filename. This lets us use
    GFile. Also only accepts binary files.

    Args:
        f: file handle
        max_num_words: number of words to load. If None, load all.

    Returns:
        Word2vec data as keyedvectors.EuclideanKeyedVectors.
    """
    header = f.readline()
    vocab_size, vector_size = (
        int(x) for x in header.rstrip().split())  # throws for invalid file format
    print("vector_size =  %d" % vector_size)
    result = gensim.models.keyedvectors.EuclideanKeyedVectors()
    num_words = 0
    result.vector_size = vector_size
    result.syn0 = np.zeros((vocab_size, vector_size), dtype=np.float32)
  
    def add_word(word, weights):
        word_id = len(result.vocab)
        if word in result.vocab:
            print("duplicate word '%s', ignoring all but first", word)
            return
        result.vocab[word] = gensim.models.keyedvectors.Vocab(
            index=word_id, count=vocab_size - word_id)
        result.syn0[word_id] = weights
        result.index2word.append(word)

    if max_num_words and max_num_words < vocab_size:
        num_embeddings = max_num_words
    else:
        num_embeddings = vocab_size
    print("Loading %d embeddings" % num_embeddings)
  
    binary_len = np.dtype(np.float32).itemsize * vector_size
    for _ in range(vocab_size):
        # mixed text and binary: read text first, then binary
        word = []
        while True:
            ch = f.read(1)
            if ch == b' ':
                break
            if ch == b'':
                raise EOFError("unexpected end of input; is count incorrect or file otherwise damaged?")
            if ch != b'\n':  # ignore newlines in front of words (some binary files have)
                word.append(ch)
        word = gensim.utils.to_unicode(b''.join(word), encoding='utf-8', errors='strict')
        weights = np.frombuffer(f.read(binary_len), dtype=np.float32)
        add_word(word, weights)
        num_words = num_words + 1
        if max_num_words and num_words == max_num_words:
            break
    if result.syn0.shape[0] != len(result.vocab):
        print(
            "duplicate words detected, shrinking matrix size from %i to %i",
            result.syn0.shape[0], len(result.vocab))
    result.syn0 = np.ascontiguousarray(result.syn0[:len(result.vocab)])
    assert (len(result.vocab), vector_size) == result.syn0.shape

    print("loaded %s matrix", result.syn0.shape)
    return result

In [4]:
%%time
# Initialize the embeddings client if this hasn't been done yet.
# For the efficiency of this notebook we just load the first 2M words, and don't
# re-initialize the client if it already exists. You could of course filter the
# word list in other ways.
if not 'client' in vars():
    print("Loading word embeddings from %s" % WORD2VEC_FILE)
    with gzip.GzipFile(fileobj=open(WORD2VEC_FILE, 'rb')) as f:
        client = load_word2vec_format(f, max_num_words=2000000)

Loading word embeddings from C:/Users/ajthampi/Workspace/Data/Interpretability/GoogleNews-vectors-negative300.bin.gz
vector_size =  300
Loading 2000000 embeddings
duplicate words detected, shrinking matrix size from %i to %i 3000000 2000000
loaded %s matrix (2000000, 300)
Wall time: 54.7 s


In [7]:
def print_knn(client, v, k, words):
    print("%d closest neighbors to A-B+C:" % k)
    count = 0
    for neighbor, score in client.similar_by_vector(v.flatten().astype(float), topn=k+3):
        if neighbor in words:
            continue
        count += 1
        if count > k:
            break
        print("%s : score=%.2f" % (neighbor, score))

In [25]:
# Use a word embedding to compute an analogy
# Edit the parameters below to get different analogies
A = "he"
B = "she"
C = "doctor"
NUM_ANALOGIES = 10

in_arr = []
for i, word in enumerate((A, B, C)):
    in_arr.append(client.word_vec(word))
in_arr = np.array([in_arr])
print_knn(client, -in_arr[0, 0, :] + in_arr[0, 1, :] + in_arr[0, 2, :],
          NUM_ANALOGIES, set([A, B, C]))

10 closest neighbors to A-B+C:
nurse : score=0.62
her : score=0.60
woman : score=0.58
mother : score=0.57
doctors : score=0.55
physician : score=0.53
pregnant : score=0.51
herself : score=0.50
sister : score=0.50
dentist : score=0.50


In [9]:
def load_analogies(filename):
    """Loads analogies.

    Args:
        filename: the file containing the analogies.

    Returns:
        A list containing the analogies.
    """
    analogies = []
    with open(filename, "r") as fast_file:
        for line in fast_file:
            line = line.strip()
            # in the analogy file, comments start with :
            if line[0] == ":":
                continue
            words = line.split()
            # there are no misformatted lines in the analogy file, so this should
            # only happen once we're done reading all analogies.
            if len(words) != 4:
                print("Invalid line: %s" % line)
                continue
            analogies.append(words)
    print("loaded %d analogies" % len(analogies))
    return analogies

In [10]:
analogies = load_analogies(ANALOGIES_FILE)
print("\n".join("%s is to %s as %s is to %s" % tuple(x) for x in analogies[:5]))

loaded 19544 analogies
Athens is to Greece as Baghdad is to Iraq
Athens is to Greece as Bangkok is to Thailand
Athens is to Greece as Beijing is to China
Athens is to Greece as Berlin is to Germany
Athens is to Greece as Bern is to Switzerland


# Defining the Protected Variable of Embeddings

The description of how to incorporate adversarial networks into machine learned models above is very generic because the technique is generally applicable for any type of systems which can be described in terms of input **X** being predictive of **Y** but potentially containing information about a protected variable **Z**. 

So long as you can construct the relevant update functions you can apply this technique. However, that doesn’t tell you much about the nature of **X**, **Y** and **Z**. 

In the case of the word analogies task above, **X=B+C−A** and **Y=D**. Figuring out what **Z** should be is a little bit trickier though.

For that we can look to a paper by [Bulokbasi et. al.](http://papers.nips.cc/paper/6227-man-is-to-computer-programmer-as-woman-is-to-homemaker-debiasing-word-embeddings) where they developed an unsupervised methodology for removing gendered semantics from word embeddings.

The first step is to select pairs of words which are relevant to the type of bias you are trying to remove. In the case of gender you can choose word pairs like “man”:”woman” and “boy”:girl” which have gender as the only difference in their semantics. Once you have these word pairs you can compute the difference between their embeddings to produce vectors in the embeddings’ semantic space which are roughly parallel to the semantics of gender. Performing Principal Components Analysis (PCA) on those vectors then gives you the major components of the semantics of gender as defined by the gendered word pairs provided.

In [11]:
def _np_normalize(v):
    """Returns the input vector, normalized."""
    return v / np.linalg.norm(v)

In [12]:
def load_vectors(client, analogies):
    """Loads and returns analogies and embeddings.

    Args:
        client: the client to query.
        analogies: a list of analogies.

    Returns:
        A tuple with:
            - the embedding matrix itself
            - a dictionary mapping from strings to their corresponding indices in the embedding matrix
            - the list of words, in the order they are found in the embedding matrix
    """
    words_unfiltered = set()
    for analogy in analogies:
        words_unfiltered.update(analogy)
    print("found %d unique words" % len(words_unfiltered))

    vecs = []
    words = []
    index_map = {}
    for word in words_unfiltered:
        try:
            vecs.append(_np_normalize(client.word_vec(word)))
            index_map[word] = len(words)
            words.append(word)
        except KeyError:
            print("word not found: %s" % word)
    print("words not filtered out: %d" % len(words))
    return np.array(vecs), index_map, words

In [13]:
%%time
embed, indices, words = load_vectors(client, analogies)
embed_dim = len(embed[0].flatten())
print("word embedding dimension: %d" % embed_dim)

found 905 unique words
words not filtered out: 905
word embedding dimension: 300
Wall time: 25 ms


In [14]:
def find_gender_direction(embed,
                          indices):
    """Finds and returns a 'gender direction'."""
    pairs = [
        ("woman", "man"),
        ("her", "his"),
        ("she", "he"),
        ("aunt", "uncle"),
        ("niece", "nephew"),
        ("daughters", "sons"),
        ("mother", "father"),
        ("daughter", "son"),
        ("granddaughter", "grandson"),
        ("girl", "boy"),
        ("stepdaughter", "stepson"),
        ("mom", "dad"),
    ]
    m = []
    for wf, wm in pairs:
        m.append(embed[indices[wf]] - embed[indices[wm]])
    m = np.array(m)

    # the next three lines are just a PCA.
    m = np.cov(np.array(m).T)
    evals, evecs = np.linalg.eig(m)
    return _np_normalize(np.real(evecs[:, np.argmax(evals)]))

In [15]:
# Using the embeddings, find the gender vector.
gender_direction = find_gender_direction(embed, indices)
print("gender direction: %s" % str(gender_direction.flatten()))

gender direction: [-7.94102545e-02 -8.62907422e-02 -9.36046141e-02 -6.82094668e-02
  2.10405590e-02  9.09854330e-03  3.69967878e-02  6.76862642e-03
 -1.35619515e-01  3.62402251e-02  6.62572962e-02 -9.91197858e-04
  4.59588244e-02 -6.13280957e-02 -2.14388396e-03 -1.15234668e-02
 -7.85432255e-02 -6.24935345e-02 -8.79221795e-02 -1.73189506e-02
  1.72767293e-02  1.72291993e-03 -3.53863145e-02  5.24758093e-03
  1.45531245e-02  7.97479856e-03 -3.40466321e-02 -2.83068031e-02
 -1.02746019e-01 -1.78703384e-01 -1.88934527e-02 -7.23408456e-02
 -1.06853716e-01 -4.48557134e-02 -5.89368438e-03 -7.94341255e-02
  1.17833274e-03 -5.86551734e-02  3.10732884e-02 -3.12044347e-03
 -4.15907990e-02 -7.17691348e-03 -2.56242192e-02 -3.41547275e-02
 -6.38683503e-02  7.83023401e-02  1.41319849e-03 -1.92862095e-02
  4.27428222e-02  6.23987503e-03  2.58495526e-02 -2.53252184e-02
 -6.58270961e-03  6.81094730e-02  2.29535994e-02 -3.01465526e-02
 -8.76619404e-03 -3.48141905e-03 -1.03283529e-02 -4.43229741e-02
  4.820

In [16]:
WORD = "she"
word_vec = client.word_vec(WORD)
print(word_vec.dot(gender_direction))

0.7064314505271143


In [17]:
words = set()
for analogy in analogies:
    for a in analogy:
        if a in indices:
            words.add(a)
df = pd.DataFrame(data={"word": list(words)})
df["gender_score"] = df["word"].map(lambda w: client.word_vec(w).dot(gender_direction))
df.sort_values(by="gender_score", inplace=True)
df.head(10)

,word,gender_score
93,his,-0.660904
211,he,-0.584860
687,unimpressive,-0.544877
797,Anaheim,-0.503144
862,Libya,-0.486664
562,playing,-0.472259
213,play,-0.459254
519,sharpest,-0.455905
758,Detroit,-0.454509
422,calmly,-0.448576


In [18]:
df.sort_values(by="gender_score", inplace=True, ascending=False)
df.head(10)

,word,gender_score
274,husband,0.950913
607,policewoman,0.816518
528,women,0.758530
678,mom,0.732802
559,princess,0.719736
336,she,0.706431
406,sisters,0.699692
481,stepson,0.686814
453,her,0.683995
875,queen,0.682354


# Adversarial Model Training

In [19]:
def tf_normalize(x):
    """Returns the input vector, normalized.

    A small number is added to the norm so that this function does not break when
    dealing with the zero vector (e.g. if the weights are zero-initialized).

    Args:
        x: the tensor to normalize
    """
    return x / (tf.norm(x) + np.finfo(np.float32).tiny)

In [20]:
class AdversarialEmbeddingModel(object):
    """A model for doing adversarial training of embedding models."""

    def __init__(self, client,
                data_p, embed_dim, projection,
                projection_dims, pred):
        """Creates a new AdversarialEmbeddingModel.

        Args:
            client: The (possibly biased) embeddings.
            data_p: Placeholder for the data.
            embed_dim: Number of dimensions used in the embeddings.
            projection: The space onto which we are "projecting".
            projection_dims: Number of dimensions of the projection.
            pred: Prediction layer.
        """
        # load the analogy vectors as well as the embeddings
        self.client = client
        self.data_p = data_p
        self.embed_dim = embed_dim
        self.projection = projection
        self.projection_dims = projection_dims
        self.pred = pred
    
    def nearest_neighbors(self, sess, in_arr, k):
        """Finds the nearest neighbors to a vector.

        Args:
            sess: Session to use.
            in_arr: Vector to find nearest neighbors to.
            k: Number of nearest neighbors to return
        Returns:
            List of up to k pairs of (word, score).
        """
        v = sess.run(self.pred, feed_dict={self.data_p: in_arr})
        return self.client.similar_by_vector(v.flatten().astype(float), topn=k)
    
    def write_to_file(self, sess, f):
        """Writes a model to disk."""
        np.savetxt(f, sess.run(self.projection))

    def read_from_file(self, sess, f):
        """Reads a model from disk."""
        loaded_projection = np.loadtxt(f).reshape([self.embed_dim, self.projection_dims])
        sess.run(self.projection.assign(loaded_projection))
    
    def fit(self, sess, data, data_p, labels, labels_p, protect, protect_p,
            gender_direction, pred_learning_rate, protect_learning_rate,
            protect_loss_weight, num_steps, batch_size, debug_interval=1000):
        """Trains a model.

        Args:
          sess: Session.
          data: Features for the training data.
          data_p: Placeholder for the features for the training data.
          labels: Labels for the training data.
          labels_p: Placeholder for the labels for the training data.
          protect: Protected variables.
          protect_p: Placeholder for the protected variables.
          gender_direction: The vector from find_gender_direction().
          pred_learning_rate: Learning rate for predicting labels.
          protect_learning_rate: Learning rate for protecting variables.
          protect_loss_weight: The constant 'alpha' found in
              debias_word_embeddings.ipynb.
          num_steps: Number of training steps.
          batch_size: Number of training examples in each step.
          debug_interval: Frequency at which to log performance metrics during
              training.
        """
        feed_dict = {
            data_p: data,
            labels_p: labels,
            protect_p: protect,
        }
        # define the prediction loss
        pred_loss = tf.losses.mean_squared_error(labels_p, self.pred)

        # compute the prediction of the protected variable.
        # The "trainable"/"not trainable" designations are for the predictor. The
        # adversary explicitly specifies its own list of weights to train.
        protect_weights = tf.get_variable("protect_weights", [self.embed_dim, 1], trainable=False)
        protect_pred = tf.matmul(self.pred, protect_weights)
        protect_loss = tf.losses.mean_squared_error(protect_p, protect_pred)

        pred_opt = tf.train.AdamOptimizer(pred_learning_rate)
        protect_opt = tf.train.AdamOptimizer(protect_learning_rate)

        protect_grad = {v: g for (g, v) in pred_opt.compute_gradients(protect_loss)}
        pred_grad = []

        # applies the gradient expression found in the document linked
        # at the top of this file.
        for (g, v) in pred_opt.compute_gradients(pred_loss):
            unit_protect = tf_normalize(protect_grad[v])
            # the two lines below can be commented out to train without debiasing
            g -= tf.reduce_sum(g * unit_protect) * unit_protect
            g -= protect_loss_weight * protect_grad[v]
            pred_grad.append((g, v))
            pred_min = pred_opt.apply_gradients(pred_grad)

        # compute the loss of the protected variable prediction.
        protect_min = protect_opt.minimize(protect_loss, var_list=[protect_weights])

        sess.run(tf.global_variables_initializer())
        sess.run(tf.local_variables_initializer())
        step = 0
        while step < num_steps:
            # pick samples at random without replacement as a minibatch
            ids = np.random.choice(len(data), batch_size, False)
            data_s, labels_s, protect_s = data[ids], labels[ids], protect[ids]
            sgd_feed_dict = {
                data_p: data_s,
                labels_p: labels_s,
                protect_p: protect_s,
            }

            if not step % debug_interval:
                metrics = [pred_loss, protect_loss, self.projection]
                metrics_o = sess.run(metrics, feed_dict=feed_dict)
                pred_loss_o, protect_loss_o, proj_o = metrics_o
            # log stats every so often: number of steps that have passed,
            # prediction loss, adversary loss
            print("step: %d; pred_loss_o: %f; protect_loss_o: %f" % (step,
                         pred_loss_o, protect_loss_o))
            for i in range(proj_o.shape[1]):
                print("proj_o: %f; dot(proj_o, gender_direction): %f)" %
                           (np.linalg.norm(proj_o[:, i]),
                           np.dot(proj_o[:, i].flatten(), gender_direction)))
            sess.run([pred_min, protect_min], feed_dict=sgd_feed_dict)
            step += 1

In [21]:
def filter_analogies(analogies,
                     index_map):
    filtered_analogies = []
    for analogy in analogies:
        missing_words = False
        for a in analogy:
            if a not in index_map:
                missing_words = True
                break
        if missing_words:
            print("at least one word missing for analogy: %s" % analogy)
        else:
            filtered_analogies.append(list(map(index_map.get, analogy)))
    return filtered_analogies

def make_data(analogies, embed, gender_direction):
    """Preps the training data.

    Args:
        analogies: a list of analogies
        embed: the embedding matrix from load_vectors
        gender_direction: the gender direction from find_gender_direction

    Returns:
        Three numpy arrays corresponding respectively to the input, output, and
        protected variables.
    """
    data = []
    labels = []
    protect = []
    for analogy in analogies:
        # the input is just the word embeddings of the first three words
        data.append(embed[analogy[:3]])
        # the output is just the word embeddings of the last word
        labels.append(embed[analogy[3]])
        # the protected variable is the gender component of the output embedding.
        # the extra pair of [] is so that the array has the right shape after
        # it is converted to a numpy array.
        protect.append([np.dot(embed[analogy[3]], gender_direction)])
    # Convert all three to numpy arrays, and return them.
    return tuple(map(np.array, (data, labels, protect)))

In [22]:
%%time
# Edit the training parameters below to experiment with different training runs.
# For example, try 
pred_learning_rate = 2**-10
protect_learning_rate = 2**-10
protect_loss_weight = 1.0
num_steps = 5000
batch_size = 3000

embed_dim = 300
projection_dims = 1

sess = tf.InteractiveSession()
with tf.variable_scope('var_scope', reuse=tf.AUTO_REUSE):
    analogy_indices = filter_analogies(analogies, indices)

    data, labels, protect = make_data(analogy_indices, embed, gender_direction)
    data_p = tf.placeholder(tf.float32, shape=[None, 3, embed_dim], name="data")
    labels_p = tf.placeholder(tf.float32, shape=[None, embed_dim], name="labels")
    protect_p = tf.placeholder(tf.float32, shape=[None, 1], name="protect")

    # projection is the space onto which we are "projecting". By default, this is
    # one-dimensional, but this can be tuned by projection_dims
    projection = tf.get_variable("projection", [embed_dim, projection_dims])

    # build the prediction layer
    # pred is the simple computation of d = -a + b + c for a : b :: c : d
    pred = -data_p[:, 0, :] + data_p[:, 1, :] + data_p[:, 2, :]
    pred -= tf.matmul(tf.matmul(pred, projection), tf.transpose(projection))

    trained_model = AdversarialEmbeddingModel(
        client, data_p, embed_dim, projection, projection_dims, pred)

    trained_model.fit(sess, data, data_p, labels, labels_p, protect, protect_p, gender_direction,
              pred_learning_rate,
            protect_learning_rate, protect_loss_weight, num_steps, batch_size)

step: 0; pred_loss_o: 0.003427; protect_loss_o: 0.015870
proj_o: 1.494562; dot(proj_o, gender_direction): 0.072412)
step: 1; pred_loss_o: 0.003427; protect_loss_o: 0.015870
proj_o: 1.494562; dot(proj_o, gender_direction): 0.072412)
step: 2; pred_loss_o: 0.003427; protect_loss_o: 0.015870
proj_o: 1.494562; dot(proj_o, gender_direction): 0.072412)
step: 3; pred_loss_o: 0.003427; protect_loss_o: 0.015870
proj_o: 1.494562; dot(proj_o, gender_direction): 0.072412)
step: 4; pred_loss_o: 0.003427; protect_loss_o: 0.015870
proj_o: 1.494562; dot(proj_o, gender_direction): 0.072412)
step: 5; pred_loss_o: 0.003427; protect_loss_o: 0.015870
proj_o: 1.494562; dot(proj_o, gender_direction): 0.072412)
step: 6; pred_loss_o: 0.003427; protect_loss_o: 0.015870
proj_o: 1.494562; dot(proj_o, gender_direction): 0.072412)
step: 7; pred_loss_o: 0.003427; protect_loss_o: 0.015870
proj_o: 1.494562; dot(proj_o, gender_direction): 0.072412)
step: 8; pred_loss_o: 0.003427; protect_loss_o: 0.015870
proj_o: 1.49456

proj_o: 1.494562; dot(proj_o, gender_direction): 0.072412)
step: 72; pred_loss_o: 0.003427; protect_loss_o: 0.015870
proj_o: 1.494562; dot(proj_o, gender_direction): 0.072412)
step: 73; pred_loss_o: 0.003427; protect_loss_o: 0.015870
proj_o: 1.494562; dot(proj_o, gender_direction): 0.072412)
step: 74; pred_loss_o: 0.003427; protect_loss_o: 0.015870
proj_o: 1.494562; dot(proj_o, gender_direction): 0.072412)
step: 75; pred_loss_o: 0.003427; protect_loss_o: 0.015870
proj_o: 1.494562; dot(proj_o, gender_direction): 0.072412)
step: 76; pred_loss_o: 0.003427; protect_loss_o: 0.015870
proj_o: 1.494562; dot(proj_o, gender_direction): 0.072412)
step: 77; pred_loss_o: 0.003427; protect_loss_o: 0.015870
proj_o: 1.494562; dot(proj_o, gender_direction): 0.072412)
step: 78; pred_loss_o: 0.003427; protect_loss_o: 0.015870
proj_o: 1.494562; dot(proj_o, gender_direction): 0.072412)
step: 79; pred_loss_o: 0.003427; protect_loss_o: 0.015870
proj_o: 1.494562; dot(proj_o, gender_direction): 0.072412)
step:

step: 146; pred_loss_o: 0.003427; protect_loss_o: 0.015870
proj_o: 1.494562; dot(proj_o, gender_direction): 0.072412)
step: 147; pred_loss_o: 0.003427; protect_loss_o: 0.015870
proj_o: 1.494562; dot(proj_o, gender_direction): 0.072412)
step: 148; pred_loss_o: 0.003427; protect_loss_o: 0.015870
proj_o: 1.494562; dot(proj_o, gender_direction): 0.072412)
step: 149; pred_loss_o: 0.003427; protect_loss_o: 0.015870
proj_o: 1.494562; dot(proj_o, gender_direction): 0.072412)
step: 150; pred_loss_o: 0.003427; protect_loss_o: 0.015870
proj_o: 1.494562; dot(proj_o, gender_direction): 0.072412)
step: 151; pred_loss_o: 0.003427; protect_loss_o: 0.015870
proj_o: 1.494562; dot(proj_o, gender_direction): 0.072412)
step: 152; pred_loss_o: 0.003427; protect_loss_o: 0.015870
proj_o: 1.494562; dot(proj_o, gender_direction): 0.072412)
step: 153; pred_loss_o: 0.003427; protect_loss_o: 0.015870
proj_o: 1.494562; dot(proj_o, gender_direction): 0.072412)
step: 154; pred_loss_o: 0.003427; protect_loss_o: 0.0158

step: 216; pred_loss_o: 0.003427; protect_loss_o: 0.015870
proj_o: 1.494562; dot(proj_o, gender_direction): 0.072412)
step: 217; pred_loss_o: 0.003427; protect_loss_o: 0.015870
proj_o: 1.494562; dot(proj_o, gender_direction): 0.072412)
step: 218; pred_loss_o: 0.003427; protect_loss_o: 0.015870
proj_o: 1.494562; dot(proj_o, gender_direction): 0.072412)
step: 219; pred_loss_o: 0.003427; protect_loss_o: 0.015870
proj_o: 1.494562; dot(proj_o, gender_direction): 0.072412)
step: 220; pred_loss_o: 0.003427; protect_loss_o: 0.015870
proj_o: 1.494562; dot(proj_o, gender_direction): 0.072412)
step: 221; pred_loss_o: 0.003427; protect_loss_o: 0.015870
proj_o: 1.494562; dot(proj_o, gender_direction): 0.072412)
step: 222; pred_loss_o: 0.003427; protect_loss_o: 0.015870
proj_o: 1.494562; dot(proj_o, gender_direction): 0.072412)
step: 223; pred_loss_o: 0.003427; protect_loss_o: 0.015870
proj_o: 1.494562; dot(proj_o, gender_direction): 0.072412)
step: 224; pred_loss_o: 0.003427; protect_loss_o: 0.0158

proj_o: 1.494562; dot(proj_o, gender_direction): 0.072412)
step: 288; pred_loss_o: 0.003427; protect_loss_o: 0.015870
proj_o: 1.494562; dot(proj_o, gender_direction): 0.072412)
step: 289; pred_loss_o: 0.003427; protect_loss_o: 0.015870
proj_o: 1.494562; dot(proj_o, gender_direction): 0.072412)
step: 290; pred_loss_o: 0.003427; protect_loss_o: 0.015870
proj_o: 1.494562; dot(proj_o, gender_direction): 0.072412)
step: 291; pred_loss_o: 0.003427; protect_loss_o: 0.015870
proj_o: 1.494562; dot(proj_o, gender_direction): 0.072412)
step: 292; pred_loss_o: 0.003427; protect_loss_o: 0.015870
proj_o: 1.494562; dot(proj_o, gender_direction): 0.072412)
step: 293; pred_loss_o: 0.003427; protect_loss_o: 0.015870
proj_o: 1.494562; dot(proj_o, gender_direction): 0.072412)
step: 294; pred_loss_o: 0.003427; protect_loss_o: 0.015870
proj_o: 1.494562; dot(proj_o, gender_direction): 0.072412)
step: 295; pred_loss_o: 0.003427; protect_loss_o: 0.015870
proj_o: 1.494562; dot(proj_o, gender_direction): 0.07241

...

step: 4697; pred_loss_o: 0.003366; protect_loss_o: 0.001537
proj_o: 0.763596; dot(proj_o, gender_direction): 0.122119)
step: 4698; pred_loss_o: 0.003366; protect_loss_o: 0.001537
proj_o: 0.763596; dot(proj_o, gender_direction): 0.122119)
step: 4699; pred_loss_o: 0.003366; protect_loss_o: 0.001537
proj_o: 0.763596; dot(proj_o, gender_direction): 0.122119)
step: 4700; pred_loss_o: 0.003366; protect_loss_o: 0.001537
proj_o: 0.763596; dot(proj_o, gender_direction): 0.122119)
step: 4701; pred_loss_o: 0.003366; protect_loss_o: 0.001537
proj_o: 0.763596; dot(proj_o, gender_direction): 0.122119)
step: 4702; pred_loss_o: 0.003366; protect_loss_o: 0.001537
proj_o: 0.763596; dot(proj_o, gender_direction): 0.122119)
step: 4703; pred_loss_o: 0.003366; protect_loss_o: 0.001537
proj_o: 0.763596; dot(proj_o, gender_direction): 0.122119)
step: 4704; pred_loss_o: 0.003366; protect_loss_o: 0.001537
proj_o: 0.763596; dot(proj_o, gender_direction): 0.122119)
step: 4705; pred_loss_o: 0.003366; protect_loss_

step: 4769; pred_loss_o: 0.003366; protect_loss_o: 0.001537
proj_o: 0.763596; dot(proj_o, gender_direction): 0.122119)
step: 4770; pred_loss_o: 0.003366; protect_loss_o: 0.001537
proj_o: 0.763596; dot(proj_o, gender_direction): 0.122119)
step: 4771; pred_loss_o: 0.003366; protect_loss_o: 0.001537
proj_o: 0.763596; dot(proj_o, gender_direction): 0.122119)
step: 4772; pred_loss_o: 0.003366; protect_loss_o: 0.001537
proj_o: 0.763596; dot(proj_o, gender_direction): 0.122119)
step: 4773; pred_loss_o: 0.003366; protect_loss_o: 0.001537
proj_o: 0.763596; dot(proj_o, gender_direction): 0.122119)
step: 4774; pred_loss_o: 0.003366; protect_loss_o: 0.001537
proj_o: 0.763596; dot(proj_o, gender_direction): 0.122119)
step: 4775; pred_loss_o: 0.003366; protect_loss_o: 0.001537
proj_o: 0.763596; dot(proj_o, gender_direction): 0.122119)
step: 4776; pred_loss_o: 0.003366; protect_loss_o: 0.001537
proj_o: 0.763596; dot(proj_o, gender_direction): 0.122119)
step: 4777; pred_loss_o: 0.003366; protect_loss_

step: 4841; pred_loss_o: 0.003366; protect_loss_o: 0.001537
proj_o: 0.763596; dot(proj_o, gender_direction): 0.122119)
step: 4842; pred_loss_o: 0.003366; protect_loss_o: 0.001537
proj_o: 0.763596; dot(proj_o, gender_direction): 0.122119)
step: 4843; pred_loss_o: 0.003366; protect_loss_o: 0.001537
proj_o: 0.763596; dot(proj_o, gender_direction): 0.122119)
step: 4844; pred_loss_o: 0.003366; protect_loss_o: 0.001537
proj_o: 0.763596; dot(proj_o, gender_direction): 0.122119)
step: 4845; pred_loss_o: 0.003366; protect_loss_o: 0.001537
proj_o: 0.763596; dot(proj_o, gender_direction): 0.122119)
step: 4846; pred_loss_o: 0.003366; protect_loss_o: 0.001537
proj_o: 0.763596; dot(proj_o, gender_direction): 0.122119)
step: 4847; pred_loss_o: 0.003366; protect_loss_o: 0.001537
proj_o: 0.763596; dot(proj_o, gender_direction): 0.122119)
step: 4848; pred_loss_o: 0.003366; protect_loss_o: 0.001537
proj_o: 0.763596; dot(proj_o, gender_direction): 0.122119)
step: 4849; pred_loss_o: 0.003366; protect_loss_

step: 4911; pred_loss_o: 0.003366; protect_loss_o: 0.001537
proj_o: 0.763596; dot(proj_o, gender_direction): 0.122119)
step: 4912; pred_loss_o: 0.003366; protect_loss_o: 0.001537
proj_o: 0.763596; dot(proj_o, gender_direction): 0.122119)
step: 4913; pred_loss_o: 0.003366; protect_loss_o: 0.001537
proj_o: 0.763596; dot(proj_o, gender_direction): 0.122119)
step: 4914; pred_loss_o: 0.003366; protect_loss_o: 0.001537
proj_o: 0.763596; dot(proj_o, gender_direction): 0.122119)
step: 4915; pred_loss_o: 0.003366; protect_loss_o: 0.001537
proj_o: 0.763596; dot(proj_o, gender_direction): 0.122119)
step: 4916; pred_loss_o: 0.003366; protect_loss_o: 0.001537
proj_o: 0.763596; dot(proj_o, gender_direction): 0.122119)
step: 4917; pred_loss_o: 0.003366; protect_loss_o: 0.001537
proj_o: 0.763596; dot(proj_o, gender_direction): 0.122119)
step: 4918; pred_loss_o: 0.003366; protect_loss_o: 0.001537
proj_o: 0.763596; dot(proj_o, gender_direction): 0.122119)
step: 4919; pred_loss_o: 0.003366; protect_loss_

step: 4983; pred_loss_o: 0.003366; protect_loss_o: 0.001537
proj_o: 0.763596; dot(proj_o, gender_direction): 0.122119)
step: 4984; pred_loss_o: 0.003366; protect_loss_o: 0.001537
proj_o: 0.763596; dot(proj_o, gender_direction): 0.122119)
step: 4985; pred_loss_o: 0.003366; protect_loss_o: 0.001537
proj_o: 0.763596; dot(proj_o, gender_direction): 0.122119)
step: 4986; pred_loss_o: 0.003366; protect_loss_o: 0.001537
proj_o: 0.763596; dot(proj_o, gender_direction): 0.122119)
step: 4987; pred_loss_o: 0.003366; protect_loss_o: 0.001537
proj_o: 0.763596; dot(proj_o, gender_direction): 0.122119)
step: 4988; pred_loss_o: 0.003366; protect_loss_o: 0.001537
proj_o: 0.763596; dot(proj_o, gender_direction): 0.122119)
step: 4989; pred_loss_o: 0.003366; protect_loss_o: 0.001537
proj_o: 0.763596; dot(proj_o, gender_direction): 0.122119)
step: 4990; pred_loss_o: 0.003366; protect_loss_o: 0.001537
proj_o: 0.763596; dot(proj_o, gender_direction): 0.122119)
step: 4991; pred_loss_o: 0.003366; protect_loss_

# Analogy Generation using Adversarial Model

In [23]:
# Parameters
A = "he"
B = "she"
C = "doctor"
NUM_ANALOGIES = 10

# Use a word embedding to compute an analogy
in_arr = []
for i, word in enumerate((A, B, C)):
    in_arr.append(client.word_vec(word))
in_arr = np.array([in_arr])
print_knn(client, sess.run(pred, feed_dict={data_p: in_arr}),
          NUM_ANALOGIES, set([A, B, C]))

10 closest neighbors to A-B+C:
nurse : score=0.68
gynecologist : score=0.67
doctors : score=0.66
nurse_practitioner : score=0.64
physician : score=0.63
pediatrician : score=0.63
pharmacist : score=0.61
dentist : score=0.61
midwife : score=0.60
dermatologist : score=0.59
